In [42]:
import pickle

import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, hstack
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm_notebook

%matplotlib inline
import seaborn as sns
from matplotlib import pyplot as plt

In [4]:
# a helper function for writing predictions to a file
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [5]:
# парсинг даты при загрузке датасета
train_df = pd.read_csv("/Users/rakhmangulovs/Downloads/train_sessions.csv",
                       index_col='session_id', parse_dates=['time1'])
test_df = pd.read_csv("/Users/rakhmangulovs/Downloads/test_sessions.csv",
                      index_col='session_id', parse_dates=['time1'])

# Sort the data by time
train_df = train_df.sort_values(by='time1')

# Look at the first rows of the training set
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55.0,2013-01-12 08:05:57,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
54843,56,2013-01-12 08:37:23,55.0,2013-01-12 08:37:23,56.0,2013-01-12 09:07:07,55.0,2013-01-12 09:07:09,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
77292,946,2013-01-12 08:50:13,946.0,2013-01-12 08:50:14,951.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:16,...,2013-01-12 08:50:16,948.0,2013-01-12 08:50:16,784.0,2013-01-12 08:50:16,949.0,2013-01-12 08:50:17,946.0,2013-01-12 08:50:17,0
114021,945,2013-01-12 08:50:17,948.0,2013-01-12 08:50:17,949.0,2013-01-12 08:50:18,948.0,2013-01-12 08:50:18,945.0,2013-01-12 08:50:18,...,2013-01-12 08:50:18,947.0,2013-01-12 08:50:19,945.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:20,0
146670,947,2013-01-12 08:50:20,950.0,2013-01-12 08:50:20,948.0,2013-01-12 08:50:20,947.0,2013-01-12 08:50:21,950.0,2013-01-12 08:50:21,...,2013-01-12 08:50:21,946.0,2013-01-12 08:50:21,951.0,2013-01-12 08:50:22,946.0,2013-01-12 08:50:22,947.0,2013-01-12 08:50:22,0


#### Transform data into format which can be fed into CountVectorizer

In [7]:
sites = ['site%s' % i for i in range(1, 11)]
train_df[sites].fillna(0).astype('int').to_csv('train_sessions_text.txt', 
                                               sep=' ', 
                       index=None, header=None)
test_df[sites].fillna(0).astype('int').to_csv('test_sessions_text.txt', 
                                              sep=' ', 
                       index=None, header=None)

In [8]:
!head -5 train_sessions_text.txt

56 55 0 0 0 0 0 0 0 0
56 55 56 55 0 0 0 0 0 0
946 946 951 946 946 945 948 784 949 946
945 948 949 948 945 946 947 945 946 946
947 950 948 947 950 952 946 951 946 947


#### Fit CountVectorizer and trasfrom data with it.

In [9]:
%%time
cv = CountVectorizer()
with open('train_sessions_text.txt') as inp_train_file:
    X_train = cv.fit_transform(inp_train_file)
with open('test_sessions_text.txt') as inp_test_file:
    X_test = cv.transform(inp_test_file)
print(X_train.shape, X_test.shape)

(253561, 41592) (82797, 41592)
CPU times: user 1.25 s, sys: 33 ms, total: 1.28 s
Wall time: 1.3 s


#### Save train targets into a separate vector

In [15]:
y_train = train_df['target'].astype('int')

# train Logistic regression

In [17]:
logit = LogisticRegression(C=1, random_state=17)

In [18]:
%%time
cv_scores = cross_val_score(logit, X_train, y_train, cv=5, scoring='roc_auc')

/opt/homebrew/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/homebrew/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i 

CPU times: user 4.8 s, sys: 167 ms, total: 4.97 s
Wall time: 5.01 s


/opt/homebrew/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [19]:
cv_scores

array([0.91412172, 0.83254011, 0.87702207, 0.89228333, 0.91380815])

In [20]:
cv_scores.mean()

0.8859550750959351

In [21]:
%%time
logit.fit(X_train, y_train)

CPU times: user 1.11 s, sys: 46.5 ms, total: 1.16 s
Wall time: 1.16 s


/opt/homebrew/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=1, random_state=17)

In [22]:
test_pred_logit1 = logit.predict_proba(X_test)[:, 1]

In [23]:
test_pred_logit1

array([2.29620120e-03, 3.44039787e-09, 1.11954354e-08, ...,
       8.71024868e-03, 4.32069363e-04, 1.86125011e-05])

In [26]:
## CV 0.885
write_to_submission_file(test_pred_logit1, 'logit_subm1.txt') ## .908 ROC AUC Public LB

In [27]:
!head logit_subm1.txt

session_id,target
1,0.002296201203563622
2,3.4403978698426815e-09
3,1.1195435411725739e-08
4,1.9673006865588265e-08
5,2.7954745786234005e-05
6,0.00021319057328538424
7,0.0006246598447373639
8,0.00012981238033936917
9,0.0008511321373508605


#### Time Features

- hour when the session started.
- morning.
- day.
- eve.
- night.

In [30]:
def add_time_features(time1_series, X_sparse):
    hour = time1_series.apply(lambda ts: ts.hour)
    morning = ((hour >= 7) & (hour <= 11)).astype('int')
    day = ((hour >= 12) & (hour <= 18)).astype('int')
    evening = ((hour >= 19) & (hour <= 23)).astype('int')
    night = ((hour >= 0) & (hour <= 6)).astype('int')
    X = hstack([X_sparse, morning.values.reshape(-1, 1), 
                day.values.reshape(-1, 1), evening.values.reshape(-1, 1), 
                night.values.reshape(-1, 1)])
    return X

In [32]:
train_df.loc[:, 'time1'].fillna(0).apply(lambda ts: ts.hour).head()

session_id
21669     8
54843     8
77292     8
114021    8
146670    8
Name: time1, dtype: int64

In [33]:
%%time
X_train_with_time = add_time_features(train_df['time1'].fillna(0), X_train)
X_test_with_time = add_time_features(test_df['time1'].fillna(0), X_test)

CPU times: user 731 ms, sys: 32.8 ms, total: 764 ms
Wall time: 787 ms


In [34]:
X_train_with_time.shape, X_test_with_time.shape

((253561, 41596), (82797, 41596))

In [37]:
#  testing model with new features
%time
cv_scores = cross_val_score(logit, X_train_with_time, y_train, cv=5, scoring='roc_auc', n_jobs=-1)

CPU times: user 4 µs, sys: 2 µs, total: 6 µs
Wall time: 35.8 µs


/opt/homebrew/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/homebrew/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i 

In [38]:
cv_scores.mean()

0.9311520367801063

In [39]:
%%time
logit.fit(X_train_with_time, y_train)

CPU times: user 1.16 s, sys: 85 ms, total: 1.25 s
Wall time: 1.29 s


/opt/homebrew/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=1, random_state=17)

In [40]:
test_pred_logit2 = logit.predict_proba(X_test_with_time)[:, 1]

In [41]:
## CV 0.931
write_to_submission_file(test_pred_logit2, 'logit_subm2.txt') ## 0.93523 ROC AUC Public LB

In [43]:
time_split = TimeSeriesSplit(n_splits=10)

In [46]:
%%time

cv_scores = cross_val_score(logit, X_train_with_time, y_train, cv=time_split, 
                            scoring='roc_auc', n_jobs=-1) 

/opt/homebrew/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/homebrew/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i 

CPU times: user 59.2 ms, sys: 53.9 ms, total: 113 ms
Wall time: 2.83 s


In [48]:
cv_scores.mean()

0.9154419811499557

#### Tune the regularization parameter "C"

In [57]:
logit = LogisticRegression(C=1, random_state=17, solver='liblinear')

In [60]:
c_values = np.logspace(-2, 2, 10)

logit_grid_searcher = GridSearchCV(estimator=logit, param_grid={'C': c_values},
                                  scoring='roc_auc', n_jobs=-1, cv=time_split, verbose=1)

In [61]:
%%time
logit_grid_searcher.fit(X_train_with_time, y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


/opt/homebrew/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


CPU times: user 1.73 s, sys: 439 ms, total: 2.17 s
Wall time: 51.3 s


GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=10, test_size=None),
             estimator=LogisticRegression(C=1, random_state=17,
                                          solver='liblinear'),
             n_jobs=-1,
             param_grid={'C': array([1.00000000e-02, 2.78255940e-02, 7.74263683e-02, 2.15443469e-01,
       5.99484250e-01, 1.66810054e+00, 4.64158883e+00, 1.29154967e+01,
       3.59381366e+01, 1.00000000e+02])},
             scoring='roc_auc', verbose=1)

In [62]:
logit_grid_searcher.best_score_, logit_grid_searcher.best_params_

(0.9155302470294512, {'C': 0.5994842503189409})

In [55]:
test_pred_logit3 = logit_grid_searcher.predict_proba(X_test_with_time)[:, 1]

In [56]:
## CV 0.915798
write_to_submission_file(test_pred_logit3, 'logit_subm3.txt') ## 0.93738 ROC AUC Public LB

In [63]:
logit = LogisticRegression(C=0.21544346900318834, random_state=17, solver='liblinear')
logit.fit(X_train_with_time, y_train)


LogisticRegression(C=0.21544346900318834, random_state=17, solver='liblinear')

In [64]:
%%time

cv_scores = cross_val_score(logit, X_train_with_time, y_train, cv=time_split, 
                            scoring='roc_auc', n_jobs=-1) 

CPU times: user 56.5 ms, sys: 110 ms, total: 167 ms
Wall time: 2.84 s


In [65]:
cv_scores.mean()

0.9135595494082471